Gaby Branin

Making It (ENAS 400)

# Calendar Break Scheduler

In [1]:
import datetime as dt
import pandas as pd
import numpy as np

### fake schedule
contains busy times, beginning of day, and end of day.

In [2]:
dt.date.today() + dt.timedelta(days=1)

datetime.date(2021, 4, 22)

In [28]:
time = dt.datetime.now()

start1 = time.replace(hour = 9, minute = 0, second = 0).timestamp()
end1 = time.replace(hour = 10, minute = 0, second = 0).timestamp()
start2 = time.replace(hour = 11, minute = 30, second = 0).timestamp()
end2 = time.replace(hour = 12, minute = 0, second = 0).timestamp()
start3 = time.replace(hour = 14, minute = 0, second = 0).timestamp()
end3 = time.replace(hour = 14, minute = 15, second = 0).timestamp()
start4 = time.replace(hour = 14, minute = 30, second = 0).timestamp()
end4 = time.replace(hour = 16, minute = 0, second = 0).timestamp()

BOD = time.replace(hour = 8, minute = 0, second = 0).timestamp()
EOD = time.replace(hour = 17, minute = 0, second = 0).timestamp()

In [4]:
busyTimes = pd.DataFrame({'start': [start1, start2, start3, start4], 'end': [end1, end2, end3, end4]})

print("User BUSY Times")
display(busyTimes)

User BUSY Times


,start,end
0,1.619096e+09,1.619100e+09
1,1.619105e+09,1.619107e+09
2,1.619114e+09,1.619115e+09
3,1.619116e+09,1.619122e+09


## find free times

In [5]:
def getFreeTimes(busy_times, user_BOD, user_EOD):  
    
#busy_times: comes from user calendar
#user_BOD: user beginning of day (input)
#user_EOD: user end of day (input)
    
    freeTimes = pd.DataFrame()
    
    #beginning of day
    if busy_times.iloc[0,0] > user_BOD:
        startFree_BOD = user_BOD
        end_first_free = busy_times.iloc[0, 0]

        first_free = pd.DataFrame({'start': [startFree_BOD] , 'end': [end_first_free]})

        freeTimes = freeTimes.append(first_free)
        
    
    #during day events
    for i in np.arange(0, len(busy_times)-1):

        startFree = busy_times.iloc[i, 1]
        endFree = busy_times.iloc[i+1, 0]

        free = pd.DataFrame({'start': [startFree] , 'end': [endFree]})

        freeTimes = freeTimes.append(free)
    
    #end of day case
    if busy_times.iloc[len(busy_times)-1, 1] < user_EOD:
        startFree_EOD = busy_times.iloc[len(busy_times)-1, 1]
        EOD = user_EOD
        
        last_free = pd.DataFrame({'start': [startFree_EOD] , 'end': [EOD]})
        freeTimes = freeTimes.append(last_free)
    
    return freeTimes

In [6]:
user_freeTimes = getFreeTimes(busyTimes, BOD, EOD)

print("User FREE times")
display(user_freeTimes)

User FREE times


,start,end
0,1.619093e+09,1.619096e+09
0,1.619100e+09,1.619105e+09
0,1.619107e+09,1.619114e+09
0,1.619115e+09,1.619116e+09
0,1.619122e+09,1.619125e+09


## schedule breaks

In [18]:
def find_breakTimes(freeTimes, break_length, time_between_breaks, time_after_busy, user_BOD):

    breakTimes = []
    
    #first break of the day:
    startBreak = user_BOD + (45*60)
    endBreak = startBreak + (break_length*60)
    one_break = [startBreak, endBreak]

    if (freeTimes.iloc[0, 0]) < startBreak < (freeTimes.iloc[0, 1]): #start of break bust be in the free time
        if (freeTimes.iloc[0, 0]) < endBreak < (freeTimes.iloc[0, 1]): #end of break bust be in the free time
            breakTimes.append(one_break)

    for i in np.arange(0,len(freeTimes)):
        #break starts user input amount of mins after beginning of free time (= end of busy time)
        startBreak = freeTimes.iloc[i, 0] + (time_after_busy*60)
        endBreak = startBreak + (break_length*60)

        possible_break = [startBreak, endBreak]

        #schedule the break if the start and the end of the break are during free time
        #possible break start must be after already scheduled break end
        
        if (possible_break[0]) > (breakTimes[0][1]): #start of possible break must be greater than end of the first manually scheduled break
            if (freeTimes.iloc[i, 0]) < startBreak < (freeTimes.iloc[i, 1]): #start of break bust be in the free time
                if (freeTimes.iloc[i, 0]) < endBreak < (freeTimes.iloc[i, 1]): #end of break bust be in the free time
                    breakTimes.append(possible_break)          
                    
        #while the end of the break is less than the end of the free time, schedule more breaks
        while (possible_break[1] < freeTimes.iloc[i,1]):
            startBreak = possible_break[1] + (time_between_breaks*60)

            #currently a 10 minute break -- should be adjustable for user input
            endBreak = startBreak + (break_length*60)

            possible_break = [startBreak, endBreak]

            #schedule break if start and end are in free period
            if (freeTimes.iloc[i, 0]) < startBreak < (freeTimes.iloc[i, 1]):
                if (freeTimes.iloc[i, 0]) < endBreak < (freeTimes.iloc[i, 1]):
                    breakTimes.append(possible_break)
                    
    return breakTimes


In [13]:
user_breakTimes = find_breakTimes(user_freeTimes, break_length=10, time_between_breaks=45, time_after_busy=15, user_BOD = BOD)
display(user_breakTimes)

[[1619095500.155701, 1619096100.155701],
 [1619100900.155701, 1619101500.155701],
 [1619104200.155701, 1619104800.155701],
 [1619108100.155701, 1619108700.155701],
 [1619111400.155701, 1619112000.155701],
 [1619122500.155701, 1619123100.155701]]

In [14]:
converted_breakTimes = []

for i in np.arange(0, len(user_breakTimes)):
    start = dt.datetime.fromtimestamp(user_breakTimes[i][0]).strftime('%Y-%m-%d %H:%M:%S')
    end = dt.datetime.fromtimestamp(user_breakTimes[i][1]).strftime('%Y-%m-%d %H:%M:%S')
    converted_breakTimes.append([start,end])
    

display(converted_breakTimes)

[['2021-04-22 12:45:00', '2021-04-22 12:55:00'],
 ['2021-04-22 14:15:00', '2021-04-22 14:25:00'],
 ['2021-04-22 15:10:00', '2021-04-22 15:20:00'],
 ['2021-04-22 16:15:00', '2021-04-22 16:25:00'],
 ['2021-04-22 17:10:00', '2021-04-22 17:20:00'],
 ['2021-04-22 20:15:00', '2021-04-22 20:25:00']]